<a href="https://colab.research.google.com/github/garrisonreid/DurhamFoodLocations/blob/master/Durham_Summer_Meals_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
!pip install pandas
!pip install folium
!pip install geopandas

In [183]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQa7xqdHPBv0l001dydV5JuDvAaUh7um07FkH3YVntOj-v43BxUcZnK3L0XgqFUXncfSA38IM77hU14/pub?gid=794529442&single=true&output=csv")
df2 = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRELikdBBJHlgSjMkGjNP6CQL5VwxrD92vVIdMDAfi58x3VHJG0dY7SjhvhGOdt-0b42b08j-YlDBY8/pub?output=csv")
df2.head()

,lng,lat,Label
0,-78.7439,35.87432,Ebenezer Church
1,-78.8161,35.88265,Pleasant Grove Church
2,-78.7478,35.88459,Raleigh Christian Life Fellowship Church
3,-78.7778,35.88737,Mount Herman Church
4,-78.9903,35.88765,Barbees Church


In [184]:
import matplotlib.pyplot as plt
!pip install descartes
!pip install plotly
# !pip install geoplot
# import geoplot.crs as gcrs

In [185]:
def make_point(row):
    return Point(row.lng, row.lat)

def make_marker(row):
  return Markers(row.Label)
  
pointsL = df.apply(make_point, axis=1)
pickupLocations = gpd.GeoDataFrame(df, geometry=pointsL)
pickupLocations.crs = {'init': 'epsg:4326'}
# pickupLocations.head()

pointsC = df2.apply(make_point, axis=1)
churchLocations = gpd.GeoDataFrame(df2, geometry=pointsC)
churchLocations.crs = {'init': 'epsg:4326'}
churchLocations.head()

,lng,lat,Label,geometry
0,-78.7439,35.87432,Ebenezer Church,POINT (-78.7439 35.87432)
1,-78.8161,35.88265,Pleasant Grove Church,POINT (-78.81610000000001 35.88265)
2,-78.7478,35.88459,Raleigh Christian Life Fellowship Church,POINT (-78.7478 35.88459)
3,-78.7778,35.88737,Mount Herman Church,POINT (-78.7778 35.88737)
4,-78.9903,35.88765,Barbees Church,POINT (-78.9903 35.88765)


In [0]:
# pickupLocations.plot()

In [0]:
# churchLocations.plot()

In [218]:
dcb = gpd.read_file("https://opendata.arcgis.com/datasets/01b3cb54b5a64105a742d5c5b4217599_0.geojson")
parks = gpd.read_file("http://gisweb2.durhamnc.gov/arcgis/rest/services/OpenData/ParksData/MapServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")

bounds = folium.GeoJson(dcb).get_bounds()
center = [(bounds[0][0] + bounds[1][0]) / 2, (bounds[0][1] + bounds[1][1]) / 2]
m = folium.Map(
  location=center,
  zoom_start=11,
  tiles='OpenStreetMap' 
)

# Add county boundary
folium.Choropleth(
    geo_data=dcb['geometry'],
    name='County Boundary',
    fill_color='#00000000',
    fill_opacity=0.0,
    line_opacity=0.8,
    line_color='gray',
    line_weight=4
).add_to(m)

# folium.LayerControl(collapsed=False).add_to(m)

In [219]:
tracts = gpd.read_file("https://raw.githubusercontent.com/garrisonreid/DurhamFoodLocations/master/durham_tracts.geojson")
tracts.head()

enrollment = pd.DataFrame(tracts)
enrollment.head()

,GEOID,GSPL,Label,geometry
0,37063001002,612,"Census Tract 10.02, Durham County, North Carolina","(POLYGON ((-78.88070399999999 35.998738, -78.8..."
1,37063001100,412,"Census Tract 11, Durham County, North Carolina","(POLYGON ((-78.89711699999999 35.990498, -78.8..."
2,37063001301,59,"Census Tract 13.01, Durham County, North Carolina","(POLYGON ((-78.90840799999999 35.97956, -78.90..."
3,37063001303,81,"Census Tract 13.03, Durham County, North Carolina","(POLYGON ((-78.908708 35.975495, -78.908700999..."
4,37063001304,536,"Census Tract 13.04, Durham County, North Carolina","(POLYGON ((-78.92167600000001 35.967415, -78.9..."


In [0]:
# folium.Choropleth(
#     geo_data=tracts['geometry'],
#     data=tracts['GSPL'],
#     columns=['GEOID', 'GSPL'],
#     key_on='feature.properties.GEOID',
#     name='Census Tracts',
#     fill_color='YlGnBu',
#     fill_opacity=0.4, # 0.8 looks good, but the data is off.
#     line_color='black',
#     line_opacity=0.4,
#     popup=enrollment['Label'],
#     legend_name='Grade school enrollment below the poverty line',
#     weight=1,
#     show=True
# ).add_to(m)

# folium.LayerControl(collapsed=False).add_to(m)

In [221]:
# Add census tracts
folium.Choropleth(
    geo_data=tracts,
    data=enrollment,
    columns=['GEOID','GSPL'],
    key_on='feature.properties.GEOID',
    fill_color='YlGnBu',
    fill_opacity=0.8,
    line_color='black',
    line_opacity=0.4,
    popup=enrollment['Label'],
    legend_name='Grade school enrollment below the poverty line',
    weight=1
).add_to(m)

# folium.LayerControl(collapsed=False).add_to(m)

In [222]:
parks = gpd.read_file("https://raw.githubusercontent.com/garrisonreid/DurhamFoodLocations/master/parks.geojson")

parkspd = pd.DataFrame(parks)
parkspd[['NAME', 'ADDRESS', 'ZIP','ACRES', 'geometry']].head()

# # Add city parks
# folium.Choropleth(
#     geo_data=parks,
#     data=parkspd['ACRES'],
#     columns=[parkspd['geometry'], parkspd['ACRES']],
#     key_on='feature.properties.ACRES',
#     name='choropleth',
#     fill_color='YlGnBu',
#     fill_opacity=0.6,
#     line_color='orange',
#     line_opacity=1.0,
#     line_weight=4
# ).add_to(m)

,NAME,ADDRESS,ZIP,ACRES,geometry
0,American Village Park,4703 AMERICAN DR,27705,4.200929,(POLYGON ((-78.97145671222768 36.0238561391778...
1,Bay-Hargrove Park,208 HARGROVE ST,27701,0.594639,(POLYGON ((-78.89667847969915 36.0045076407773...
2,Belmont Park,2207 SOVEREIGN ST,27705,0.497245,(POLYGON ((-78.92756412637395 36.0273921181991...
3,Bethesda Park,1814 STAGE RD,27703,20.511371,(POLYGON ((-78.84626399613565 35.9459431788064...
4,Birchwood Park,3105 HURSEY ST,27703,4.908871,(POLYGON ((-78.84216114254727 35.9812481082154...


In [0]:
# # Red markers are churches
for index, row in churchLocations.iterrows():
  folium.CircleMarker([row['lat'], row['lng']], radius=4, popup=row['Label'],color='red', fill_color='red').add_to(m) #, fill_opacity=0.2, line_opacity=0.4

# # Blue markers are 2018 pickup locations
for index, row in pickupLocations.iterrows():
  folium.CircleMarker([row['lat'], row['lng']], radius=4, popup=row['Label'],color='blue', fill_color='blue').add_to(m)

In [224]:
edci = gpd.read_file("https://raw.githubusercontent.com/garrisonreid/DurhamFoodLocations/master/EDCI_Zone.geojson")
edci.head()

# # Add EDCI Zone boundary
folium.Choropleth(
    geo_data=edci['geometry'],
    name='EDCI',
    fill_color='yellow',
    fill_opacity=0.6,
    line_color='orange',
    line_opacity=1.0,
    line_weight=4,
#     show=True,
).add_to(m)

# folium.LayerControl(collapsed=False).add_to(m)

In [225]:
m